In [2]:
#Reading the data
import pandas as pd
data = pd.read_csv("Amazon Sale Report.csv",encoding='latin1')

C:\Users\user\AppData\Local\Temp\ipykernel_3364\1461423437.py:3: DtypeWarning: Columns (21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Amazon Sale Report.csv",encoding='latin1')


# Step2: Data Preprocessing

In [3]:
import numpy as np
from scipy.stats import zscore

### Handling missing values


In [4]:

for column in data.columns:
    if data[column].dtype == 'float64':
        data[column].fillna(data[column].mean(), inplace=True)
    elif data[column].dtype == 'object':
        data[column].fillna(data[column].mode()[0], inplace=True)


### Data type Conversion

In [5]:
data.dtypes

index                 float64
Order ID               object
Date                   object
Status                 object
Fulfilment             object
Sales Channel          object
ship-service-level     object
Style                  object
SKU                    object
Category               object
Size                   object
ASIN                   object
Courier Status         object
Qty                   float64
currency               object
Amount                float64
ship-city              object
ship-state             object
ship-postal-code      float64
ship-country           object
promotion-ids          object
B2B                      bool
fulfilled-by           object
Unnamed: 22              bool
dtype: object

In [6]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

C:\Users\user\AppData\Local\Temp\ipykernel_3364\3064529874.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Date'] = pd.to_datetime(data['Date'], errors='coerce')


In [7]:
data['index'] = data['index'].astype(int)
data['Qty'] = data['Qty'].astype(int)
data['Amount'] = data['Amount'].astype(int)
data['ship-postal-code'] = data['ship-postal-code'].astype(int)

In [8]:
data.dtypes

index                          int32
Order ID                      object
Date                  datetime64[ns]
Status                        object
Fulfilment                    object
Sales Channel                 object
ship-service-level            object
Style                         object
SKU                           object
Category                      object
Size                          object
ASIN                          object
Courier Status                object
Qty                            int32
currency                      object
Amount                         int32
ship-city                     object
ship-state                    object
ship-postal-code               int32
ship-country                  object
promotion-ids                 object
B2B                             bool
fulfilled-by                  object
Unnamed: 22                     bool
dtype: object

### Handling Outliers

In [9]:
numerical_columns = data.select_dtypes(include=['int32']).columns

In [10]:
for column in numerical_columns:
    data['zscore'] = zscore(data[column])
    outliers = data[np.abs(data['zscore']) > 3]  # Threshold of 3 standard deviations
    print(f"Outliers in {column}:\n", outliers[[column, 'zscore']])
    # Remove outliers
    data = data[np.abs(data['zscore']) <= 3]

# Drop the zscore column after outlier treatment
data.drop(columns=['zscore'], inplace=True)

Outliers in index:
 Empty DataFrame
Columns: [index, zscore]
Index: []
Outliers in Qty:
         Qty     zscore
265       2   3.496130
647       2   3.496130
937      15  44.980613
1018      2   3.496130
1057      2   3.496130
...     ...        ...
126784    2   3.496130
127401    2   3.496130
128406    2   3.496130
128705    2   3.496130
128958    2   3.496130

[388 rows x 2 columns]
Outliers in Amount:
         Amount    zscore
299       1463  3.036635
324       1902  4.669541
349       1999  5.030342
398       1463  3.036635
416       1499  3.170541
...        ...       ...
128345    1559  3.393717
128404    1556  3.382558
128465    1473  3.073831
128667    1477  3.088709
128875    1523  3.259811

[910 rows x 2 columns]
Outliers in ship-postal-code:
 Empty DataFrame
Columns: [ship-postal-code, zscore]
Index: []


In [14]:
# Remove leading and trailing spaces from column names
clean_columns = [column.strip() for column in data.columns]

# Remove leading and trailing spaces from data rows
clean_data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [15]:
clean_data.to_csv('Cleaned_Amazon_Sale_Report.csv', index=False)